<a href="https://colab.research.google.com/github/ChVenkatSai/Sampling-for-LLM/blob/main/LoRA_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 40.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
# Tokenize dataset
class GPT2Dataset(Dataset):
    def __init__(self, tokenizer, dataset, num_samples, split="train"):
        self.examples = tokenizer(dataset[split]["text"][:num_samples], padding=True, truncation=True, return_tensors="pt")

    def __len__(self):
        return len(self.examples["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.examples.items()}

In [ ]:
from torch.cuda.amp import autocast, GradScaler
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
import numpy as np
import torch.nn.functional as F

In [ ]:
lora_config = LoraConfig(
    r=8,  # Set the rank factor
    lora_alpha=16,  # Set the alpha factor
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
    init_lora_weights="gaussian",  # Initialize LoRA weights using Gaussian distribution
    target_modules=["lm_head"]  # Specify the target attention module for LoRA
)



In [ ]:
train_dataset = GPT2Dataset(tokenizer, dataset, 1001)
test_dataset = GPT2Dataset(tokenizer, dataset, 101, "test")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Linear,  torch.nn.Conv1d)):
        print(name)
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

lm_head


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 408,200 || all params: 82,320,776 || trainable%: 0.49586510214626744


In [ ]:
batch_size = 2
epochs = 2
learning_rate = 0.001
data_size = len(train_dataset)
num_batch = data_size//batch_size + 1
T = epochs*num_batch
M = 2
temperature = 0.00001
alpha = 0.7

In [ ]:
from torch.optim.lr_scheduler import _LRScheduler
class CosineLR(_LRScheduler):
    def __init__(self, optimizer, T, M, lr_0, num_batch, last_epoch=-1):
        self.T = T
        self.M = M
        self.lr_0 = lr_0
        self.num_batch = num_batch
        super(CosineLR, self).__init__(optimizer, last_epoch)

    def step(self, epoch=None, batch_idx=None):
        if epoch is None or batch_idx is None:
              print('Breaking')
              return
        rcounter = epoch*self.num_batch+batch_idx
        cos_inner = np.pi * (rcounter % (self.T // self.M))
        cos_inner /= self.T // self.M
        cos_out = np.cos(cos_inner) + 1
        for param_group in self.optimizer.param_groups:
            lr = 0.5 * (cos_out) * self.lr_0
            param_group['lr'] = lr

In [ ]:


# Optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Custom training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

scaler = GradScaler()

for epoch in range(epochs):
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        optimizer.zero_grad()


        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 100 == 0:
            print('Batch [{}/{}], Loss: {:.3f}'.format(batch_idx + 1, len(train_loader), total_loss / (batch_idx + 1)))

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {average_loss}")

torch.save(model.state_dict(), "fine_tuned_model.pt")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch [1/501], Loss: 8.078
Batch [101/501], Loss: 2.264
Batch [201/501], Loss: 1.485
Batch [301/501], Loss: 1.198
Batch [401/501], Loss: 1.035


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch [501/501], Loss: 0.931
Epoch 1/2, Average Loss: 0.9309421964194217
Batch [1/501], Loss: 0.926
Batch [101/501], Loss: 0.548
Batch [201/501], Loss: 0.540
Batch [301/501], Loss: 0.530
Batch [401/501], Loss: 0.531
Batch [501/501], Loss: 0.528
Epoch 2/2, Average Loss: 0.5281764777663522


In [ ]:

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Variables to track loss
total_loss = 0
num_batches = len(test_loader)
ensemble_predictions = []
total_tokens = 0
correct_tokens = 0
total_entropy = 0

# Evaluation mode with dropout enabled
model.train()  # Set model to training mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prediction loop
predictions = []
with torch.no_grad():
    model.load_state_dict(torch.load(f'fine_tuned_model.pt'))
    for batch_idx, batch in enumerate(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        logits = outputs.logits
        predicted_tokens = torch.argmax(logits, dim=-1)
        correct_tokens += (predicted_tokens == input_ids).sum().item()
        total_tokens += input_ids.numel()

        total_loss += outputs.loss.item()

        entropy_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), input_ids.view(-1))
        total_entropy += entropy_loss.item()

        if batch_idx%10 == 0:
          print('Batch [{}/{}], Loss: {:.3f}'.format(batch_idx + 1, len(test_loader), total_loss / (batch_idx + 1)))

perplexity = torch.exp(torch.tensor(total_loss / len(test_loader)))
token_level_accuracy = correct_tokens / total_tokens

average_loss = total_loss / (num_batches)
average_entropy = total_entropy / len(test_loader)

# Print test loss
print(f"Test Loss: {average_loss}")
print(f"Entropy Loss: {average_entropy}")
print(f"Perplexity: {perplexity.item()}")
print(f"Token-level Accuracy: {token_level_accuracy}")

Batch [1/51], Loss: 0.062
Batch [11/51], Loss: 0.934
Batch [21/51], Loss: 0.682
Batch [31/51], Loss: 0.835
Batch [41/51], Loss: 0.800
Batch [51/51], Loss: 0.928
Test Loss: 0.9282681956321147
Entropy Loss: 1.7478633313622398
Perplexity: 2.530123710632324
Token-level Accuracy: 0.8030107843498104
